# US Immigration - Data Analysis
### Data Engineering Capstone Project
#### Created by Nicolas Nouchi

#### Project Summary

In this project, we will be analyzing the immigration volumes in US cities by month and year. This includes creating an ETL pipeline that includes assessing the initial data from CSV files and text files. The data will be transformed into clean, actionable datasets. 

This information can be used by a government department/agency or an airline company to understand the distribution of visitors within a city, allowing them to make improvements of the infrastructure of their airports or anticpate the need for further housing developments.

This information is distributed between Amazon Web Services tools **S3 and Redshift**.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import configparser
import psycopg2
import boto3
#from sql_queries import create_table_queries, drop_table_queries, copy_table_queries

In [2]:
#read data
fname='../../data2/GlobalLandTemperaturesByCity.csv'
global_temp_df = pd.read_csv(fname)

In [3]:
global_temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [4]:
fname = '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
immigration_df = pd.read_sas(fname, 'sas7bdat', encoding="ISO-8859-1")
immigration_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [5]:
airport_codes = pd.read_csv('airport-codes_csv.csv')
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [6]:
us_demographics = pd.read_csv('us-cities-demographics.csv',delimiter=';')
us_demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [7]:
immigration_data_sample = pd.read_csv('immigration_data_sample.csv',delimiter=',').drop('Unnamed: 0',axis=1)
immigration_data_sample.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


## Step 1: Scope the Project and Gather Data

### Scope
- **Scope of the Project**
    - In this project, I have planned to analyze the immigration volumes in the US cities by month and year. The data I plan on using is listed below (**Data Selection**) and will utilize the pandas library to clean the CSV & TXT files. Then I will import these cleaned csv files into S3 and reference them to create my tables in Redshift.  

## Describe and Gather Data 

- **Data Selection**
    - I94 Immigration Data: This data comes from the US National Tourism and Trade Office. It includes various information regarding the year, month, port, birth year and more of an individual.

    - World Temperature Data: This dataset came from Kaggle. It includes various information regarding the average temperature, year, city and country.

    - U.S. City Demographic Data: This data comes from OpenSoft. It includes various information regarding the city, state, median age, total population and veterans.

    - Airport Code Table: This is a simple table of airport codes and corresponding cities. It includes various information regarding the type of airport, the elevation, iso_region and name of the airport.

- **SAS Labels Descriptions**
    - I94_ADDRL: This includes states and territories from the United States
    - I94_CNTYL: This includes countries and their country code (i.e. i94res from the immigration table)  
    - I94_MODE: This includes transportation modes of travel and their designated code (i.e. i94mode from the immigration table)
    - I94_PORT: This includes port names and codes (i.e. i94port from the immigration table)
    - I94_VISA: This includes visa codes and their type (i.e. i94visa from the immigration table) 

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = "/opt/conda/bin:/opt/spark-2.4.3-bin-hadoop2.7/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/lib/jvm/java-8-openjdk-amd64/bin"
os.environ["SPARK_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"
os.environ["HADOOP_HOME"] = "/opt/spark-2.4.3-bin-hadoop2.7"

In [9]:
from pyspark.sql import SparkSession

In [10]:
# Read in the data here
spark = SparkSession.builder.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").getOrCreate()

df_spark =spark.read.load('./sas_data')
print("Completed, ran successfully")


Completed, ran successfully


In [11]:
df_spark.head()

Row(cicid=5748517.0, i94yr=2016.0, i94mon=4.0, i94cit=245.0, i94res=438.0, i94port='LOS', arrdate=20574.0, i94mode=1.0, i94addr='CA', depdate=20582.0, i94bir=40.0, i94visa=1.0, count=1.0, dtadfile='20160430', visapost='SYD', occup=None, entdepa='G', entdepd='O', entdepu=None, matflag='M', biryear=1976.0, dtaddto='10292016', gender='F', insnum=None, airline='QF', admnum=94953870030.0, fltno='00011', visatype='B1')

In [12]:
#write to parquet
#df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [13]:
#Remove the quotations and empty space from the code and port text file
i94port_df = pd.read_csv('SAS_dimensions/I94_PRTL.txt',header=None,sep="	=	",names=['code','port'])
i94port_df['code'] = i94port_df['code'].map(lambda x: x.strip("'"))
i94port_df['port'] = i94port_df['port'].map(lambda x: x.strip("'"))
i94port_df['port'] = i94port_df['port'].str.lstrip()
i94port_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,code,port
0,ALC,"ALCAN, AK"
1,ANC,"ANCHORAGE, AK"
2,BAR,"BAKER AAF - BAKER ISLAND, AK"
3,DAC,"DALTONS CACHE, AK"
4,PIZ,"DEW STATION PT LAY DEW, AK"


In [14]:
#Move clean port data to a new csv file
i94port_df.to_csv('SAS_dimensions_cleaned/I94_PORT.csv',index=False,mode='w')

In [15]:
i94_visa_df = pd.read_csv('SAS_dimensions/I94_VISA.txt',header=None,sep=" = ",names=['value','cd'])
i94_visa_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,value,cd
0,1,Business
1,2,Pleasure
2,3,Student


In [16]:
#Move clean visa data to a new csv file
i94_visa_df.to_csv('SAS_dimensions_cleaned/I94_VISA.csv',index=False,mode='w')

In [17]:
# Remove the quotations from the country column 
i94cntyl_df = pd.read_csv('SAS_dimensions/I94_CNTYL.txt',header=None,sep=" =  ",names=['code','country'])
i94cntyl_df['country'] = i94cntyl_df['country'].map(lambda x: x.strip("'"))
i94cntyl_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,code,country
0,582,"MEXICO Air Sea, and Not Reported (I-94, no lan..."
1,236,AFGHANISTAN
2,101,ALBANIA
3,316,ALGERIA
4,102,ANDORRA


In [18]:
#Move clean country data to a new csv file
i94cntyl_df.to_csv('SAS_dimensions_cleaned/I94_CNTYL.csv',index=False,mode='w')

In [19]:
#Remove the quotations from the mode column
i94mode_df = pd.read_csv('SAS_dimensions/I94_MODE.txt',header=None,sep=" = ",names=["code","mode"])
i94mode_df['mode'] = i94mode_df['mode'].map(lambda x: x.strip("'"))
i94mode_df.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,code,mode
0,1,Air
1,2,Sea
2,3,Land
3,9,Not reported


In [20]:
#Move mode data to a new csv file
i94mode_df.to_csv('SAS_dimensions_cleaned/I94_MODE.csv',index=False,mode='w')

In [21]:
#Remove the quotations from the code & addr columns
i94addr_df = pd.read_csv("SAS_dimensions/I94_ADDRL.txt",header=None,sep="=",names=["code","addr"])
i94addr_df['code'] = i94addr_df['code'].map(lambda x: x.strip("'"))
i94addr_df['addr'] = i94addr_df['addr'].map(lambda x: x.strip("'"))
i94addr_df.head()

,code,addr
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [22]:
#Move addr data to a new csv file
i94addr_df.to_csv('SAS_dimensions_cleaned/I94_ADDRL.csv',index=False,mode='w')

In [23]:
immigration_df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [24]:
immigration_df['i94cit'].unique()

array([ 692.,  254.,  101.,  102.,  103.,  104.,  105.,  107.,  108.,
        109.,  110.,  111.,  112.,  113.,  114.,  115.,  116.,  117.,
        118.,  119.,  121.,  122.,  123.,  124.,  126.,  127.,  128.,
        129.,  130.,  131.,  133.,  135.,  140.,  141.,  145.,  147.,
        148.,  151.,  152.,  153.,  154.,  155.,  156.,  157.,  159.,
        162.,  163.,  164.,  165.,  166.,  167.,  201.,  203.,  204.,
        206.,  207.,  209.,  213.,  218.,  220.,  236.,  242.,  244.,
        245.,  249.,  250.,  251.,  253.,  255.,  256.,  257.,  258.,
        260.,  261.,  262.,  263.,  266.,  267.,  268.,  272.,  273.,
        274.,  296.,  297.,  298.,  299.,  301.,  304.,  310.,  311.,
        315.,  316.,  320.,  323.,  324.,  326.,  329.,  332.,  336.,
        338.,  339.,  340.,  342.,  343.,  344.,  345.,  348.,  350.,
        352.,  368.,  369.,  370.,  371.,  372.,  373.,  375.,  376.,
        382.,  383.,  384.,  385.,  386.,  387.,  388.,  389.,  390.,
        391.,  392.,

In [25]:
immigration_df.drop(immigration_df[immigration_df['i94port']=='XXX'].index,inplace=True)

In [26]:
immigration_df['i94port'].unique()

array(['ATL', 'WAS', 'NYC', 'TOR', 'BOS', 'HOU', 'MIA', 'CHI', 'LOS',
       'CLT', 'DEN', 'DAL', 'DET', 'NEW', 'FTL', 'LVG', 'ORL', 'NOL',
       'PIT', 'SFR', 'SPM', 'POO', 'PHI', 'SEA', 'SLC', 'TAM', 'HAM',
       'NAS', 'VCV', 'MAA', 'AUS', 'HHW', 'OGG', 'PHO', 'SDP', 'SFB',
       'EDA', 'MON', 'CLG', 'DUB', 'FMY', 'YGF', 'SAJ', 'CIN', 'BAL',
       'RDU', 'WPB', 'STT', 'OAK', 'NSV', 'SNA', 'OTT', 'X96', '5KE',
       'CLE', 'HAR', 'PSP', 'CHR', 'HAL', 'SAA', 'KOA', 'SHA', 'WIN',
       'BGM', 'NCA', 'OPF', 'SAI', 'JFA', 'AGA', 'ONT', 'CLM', 'STL',
       'W55', 'CHS', 'SNJ', 'SRQ', 'ANC', 'LNB', 'LIH', 'MIL', 'INP',
       'KAN', 'ROC', 'SAC', 'BRO', 'LAR', 'RNO', 'SGR', 'ELP', 'MCA',
       'MDT', 'SPE', 'FPR', 'SYR', 'ICT', 'MLB', 'ADS', 'TUC', 'DLR',
       'CAE', 'CHA', 'HSV', 'WIL', 'HPN', 'HEF', 'BRG', 'BED', 'DAB',
       'JAC', 'FRB', 'SWF', 'KEY', 'PTK', 'MWH', 'X44', 'MYR', 'APF',
       'ATW', 'PVD', 'BUF', 'PIE', 'MHT', 'BDL', 'NYL', 'VNY', '5T6',
       'LEX', 'NOR',

In [27]:
#specify the required columns for the immigration table
immigration_df = immigration_df[['cicid','i94yr','i94mon','i94cit','i94res','i94port','arrdate','i94mode','i94addr','depdate','i94bir','i94visa','count','visapost','biryear','gender','airline','visatype']]
immigration_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,visapost,biryear,gender,airline,visatype
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,25.0,3.0,1.0,SEO,1991.0,M,NaN,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,55.0,2.0,1.0,NaN,1961.0,M,OS,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,28.0,2.0,1.0,NaN,1988.0,NaN,AA,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,4.0,2.0,1.0,NaN,2012.0,NaN,AA,B2
5,18.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MI,20555.0,57.0,1.0,1.0,NaN,1959.0,NaN,AZ,B1


In [28]:
#Drop any NaN values
immigration_df = immigration_df.dropna()
immigration_df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,visapost,biryear,gender,airline,visatype
12,27.0,2016.0,4.0,101.0,101.0,BOS,20545.0,1.0,MA,20549.0,58.0,1.0,1.0,TIA,1958.0,M,LH,B1
13,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,56.0,1.0,1.0,TIA,1960.0,F,LH,B1
14,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,62.0,2.0,1.0,TIA,1954.0,M,AZ,B2
15,30.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NJ,20578.0,49.0,2.0,1.0,TIA,1967.0,M,OS,B2
16,31.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NY,20611.0,43.0,2.0,1.0,TIA,1973.0,M,OS,B2


In [29]:
#Drop any duplicate values
immigration_df['cicid'].drop_duplicates()
immigration_df.count()
immigration_df.head(10)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,visapost,biryear,gender,airline,visatype
12,27.0,2016.0,4.0,101.0,101.0,BOS,20545.0,1.0,MA,20549.0,58.0,1.0,1.0,TIA,1958.0,M,LH,B1
13,28.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20549.0,56.0,1.0,1.0,TIA,1960.0,F,LH,B1
14,29.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,MA,20561.0,62.0,2.0,1.0,TIA,1954.0,M,AZ,B2
15,30.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NJ,20578.0,49.0,2.0,1.0,TIA,1967.0,M,OS,B2
16,31.0,2016.0,4.0,101.0,101.0,ATL,20545.0,1.0,NY,20611.0,43.0,2.0,1.0,TIA,1973.0,M,OS,B2
17,33.0,2016.0,4.0,101.0,101.0,HOU,20545.0,1.0,TX,20554.0,53.0,2.0,1.0,TIA,1963.0,F,TK,B2
20,36.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20561.0,37.0,2.0,1.0,TIA,1979.0,M,TK,B2
21,37.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NJ,20567.0,49.0,2.0,1.0,TIA,1967.0,F,AZ,B2
22,38.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,NY,20575.0,33.0,2.0,1.0,TIA,1983.0,M,AZ,B2
23,39.0,2016.0,4.0,101.0,101.0,MIA,20545.0,1.0,FL,20574.0,65.0,2.0,1.0,TIA,1951.0,F,TK,B2


In [30]:
#Move the immigration data to a clean CSV file
immigration_df.to_csv('immigration_df_final.csv',index=False,mode='w')
print('immigration_df was moved to a CSV file')

immigration_df was moved to a CSV file


In [31]:
global_temp_df = global_temp_df.dropna()

In [32]:
global_temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E


In [33]:
# Move the global temperature data to a new CSV file
global_temp_df.to_csv('global_temp_final.csv',index=False,mode='w')
print('global_temp_df was moved to a CSV file')

global_temp_df was moved to a CSV file


In [34]:
us_demographics = us_demographics.drop_duplicates()
print('Duplicates dropped')
us_demographics = us_demographics.dropna()
print('Null values dropped')

Duplicates dropped
Null values dropped


In [35]:
us_demographics.count()

City                      2875
State                     2875
Median Age                2875
Male Population           2875
Female Population         2875
Total Population          2875
Number of Veterans        2875
Foreign-born              2875
Average Household Size    2875
State Code                2875
Race                      2875
Count                     2875
dtype: int64

In [36]:
#us_demographics.head()
us_demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2875 entries, 0 to 2890
Data columns (total 12 columns):
City                      2875 non-null object
State                     2875 non-null object
Median Age                2875 non-null float64
Male Population           2875 non-null float64
Female Population         2875 non-null float64
Total Population          2875 non-null int64
Number of Veterans        2875 non-null float64
Foreign-born              2875 non-null float64
Average Household Size    2875 non-null float64
State Code                2875 non-null object
Race                      2875 non-null object
Count                     2875 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 292.0+ KB


In [37]:
#Move the US demographic data to a new csv file
us_demographics.to_csv('us_demographics_final.csv',index=False,mode='w')

In [38]:
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [39]:
#split coordinates into latitude & Longitude using lamdba
airport_codes['latitude'] = airport_codes['coordinates'].apply(lambda x: x.split(',')[0])
airport_codes['longitude'] = airport_codes['coordinates'].apply(lambda x: x.split(',')[1])

In [40]:
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates,latitude,longitude
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125",-74.93360137939453,40.07080078125
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022",-101.473911,38.704022
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968",-151.695999146,59.94919968
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172",-86.77030181884766,34.86479949951172
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087",-91.254898,35.6087


In [41]:
airport_codes = airport_codes[['ident','type','name','elevation_ft','continent','iso_country','iso_region','municipality','gps_code','iata_code','local_code','latitude','longitude']]
airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 13 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
latitude        55075 non-null object
longitude       55075 non-null object
dtypes: float64(1), object(12)
memory usage: 5.5+ MB


In [42]:
#Change the type of the latitude and longitude columns to float type and round to 2 decimal points
airport_codes['latitude'] = airport_codes['latitude'].astype(float).round(2)
airport_codes['longitude'] = airport_codes['longitude'].astype(float).round(2)
airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 13 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
latitude        55075 non-null float64
longitude       55075 non-null float64
dtypes: float64(3), object(10)
memory usage: 5.5+ MB


In [43]:
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,latitude,longitude
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,-74.93,40.07
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,-101.47,38.70
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,-151.70,59.95
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,-86.77,34.86
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,-91.25,35.61


In [44]:
#Verify the continents located and what is missing from the table
airport_codes['continent'].unique()

array([nan, 'OC', 'AF', 'AN', 'EU', 'AS', 'SA'], dtype=object)

In [45]:
#Replace the missing values of the continent with NA (North America)
airport_codes['continent'] = airport_codes['continent'].astype(str).apply(lambda x: x.replace('nan','NA'))

In [46]:
airport_codes[airport_codes['continent'] == 'nan'].count()

ident           0
type            0
name            0
elevation_ft    0
continent       0
iso_country     0
iso_region      0
municipality    0
gps_code        0
iata_code       0
local_code      0
latitude        0
longitude       0
dtype: int64

In [47]:
airport_codes.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,latitude,longitude
0,00A,heliport,Total Rf Heliport,11.0,NA,US,US-PA,Bensalem,00A,NaN,00A,-74.93,40.07
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NA,US,US-KS,Leoti,00AA,NaN,00AA,-101.47,38.70
2,00AK,small_airport,Lowell Field,450.0,NA,US,US-AK,Anchor Point,00AK,NaN,00AK,-151.70,59.95
3,00AL,small_airport,Epps Airpark,820.0,NA,US,US-AL,Harvest,00AL,NaN,00AL,-86.77,34.86
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NA,US,US-AR,Newport,NaN,NaN,NaN,-91.25,35.61


In [48]:
#Check to see if any iata_codes utilize US or USA as their code
airport_codes[airport_codes['iata_code'] =='USA']
#airport_codes[airport_codes['iata_code'] =='US']

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,latitude,longitude
27692,KJQF,small_airport,Concord-Padgett Regional Airport,705.0,NA,US,US-NC,Concord,KJQF,USA,JQF,-80.71,35.39


In [49]:
airport_codes['iso_country'].unique()

array(['US', 'PR', 'MH', 'MP', 'GU', 'SO', 'AQ', 'GB', 'PG', 'AD', 'SD',
       'SA', 'AE', 'SS', 'ES', 'CN', 'AF', 'LK', 'SB', 'CO', 'AU', 'MG',
       'TD', 'AL', 'AM', 'MX', 'MZ', 'PW', 'NR', 'AO', 'AR', 'AS', 'AT',
       'ZZ', 'GA', 'AZ', 'BA', 'BB', 'BE', 'DE', 'BF', 'BG', 'GL', 'BH',
       'BI', 'IS', 'BJ', 'OM', 'XK', 'BM', 'KE', 'PH', 'BO', 'BR', 'BS',
       'CV', 'BW', 'FJ', 'BY', 'UA', 'LR', 'BZ', 'CA', 'CD', 'CF', 'CG',
       'MR', 'CH', 'CL', 'CM', 'MA', 'CR', 'CU', 'CY', 'CZ', 'SK', 'PA',
       'DZ', 'ID', 'GH', 'RU', 'CI', 'DK', 'NG', 'DO', 'NE', 'HR', 'TN',
       'TG', 'EC', 'EE', 'FI', 'EG', 'GG', 'JE', 'IM', 'FK', 'EH', 'NL',
       'IE', 'FO', 'LU', 'NO', 'PL', 'ER', 'MN', 'PT', 'SE', 'ET', 'LV',
       'LT', 'ZA', 'SZ', 'GQ', 'SH', 'MU', 'IO', 'ZM', 'FM', 'KM', 'YT',
       'RE', 'TF', 'ST', 'FR', 'SC', 'ZW', 'MW', 'LS', nan, 'ML', 'GM',
       'GE', 'GF', 'SL', 'GW', 'GN', 'SN', 'GR', 'GT', 'TZ', 'GY', 'SR',
       'DJ', 'HK', 'LY', 'HN', 'VN', 'KZ', 'RW', 'HT

In [50]:
airport_codes['iata_code'].unique()

array([nan, 'UTK', 'OCA', ..., 'SHE', 'YNJ', 'YKH'], dtype=object)

In [51]:
#Replace iata_code that has missing values with None
airport_codes['iata_code'] = airport_codes['iata_code'].astype(str).apply(lambda x: x.replace('nan','None'))


In [52]:
airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55075 entries, 0 to 55074
Data columns (total 13 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       55075 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       55075 non-null object
local_code      28686 non-null object
latitude        55075 non-null float64
longitude       55075 non-null float64
dtypes: float64(3), object(10)
memory usage: 5.5+ MB


In [53]:
airport_codes= airport_codes.drop_duplicates()

In [54]:
#Write the airport data to a new csv file
airport_codes.to_csv('airport_codes_final.csv',index=False,mode='w')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

**3.1**

The best model to perform high-value analysis in this situation would be to utilize a Star Schema. 

This model would focus on utilizing one fact table *i94_immigration_data* and multiple dimensional tables including us_demographic_data, airport_data, global_temp_data, i94_cntyl_data, i94_port_data, i94_mode_data, i94_addr_data, and i94_visa_data.

The flexibility is there for these tables to be utilized in additional analysis that may be required beyond the fact table, therefore if needed, one could join together these tables. Then they can utilize the information necessary such as how many people traveled to a given city within a period of months or years.


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [55]:
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries, insert_table_queries

#drop tables if they exist
def drop_tables(cur, conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()


def create_tables(cur, conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()
    
        
def insert_tables(cur,conn):
    for query in insert_table_queries:
        cur.execute(query)
        conn.commit()


def main():
    config = configparser.ConfigParser()
    config.read('dl.cfg')

    conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()

    drop_tables(cur, conn)
    print('Droping tables is complete')
    create_tables(cur, conn)
    print('Creating tables is complete')
    insert_tables(cur,conn)
    print('Inserting values from S3 to Redshift is complete')
    
    
    print('\nSuccess')

    conn.close()


if __name__ == "__main__":
    main()

Droping tables is complete
Creating tables is complete
Inserting values from S3 to Redshift is complete

Success


#### 4.2 Data Quality Checks


- 1) Ensuring that the primary keys of the tables align with the source data, I will compare the number of unique rows and if they are equivalent, then the check will show that there are correct constraints.
- 2) The original dataframe should show for all tables that have a primary key, that the unique values align with the total count of the primary key. This is done by the function **check_unique_counts** that compares the original dataframe from the csv file and the table hosted in Redshift.
- 3) Checking datatypes of the tables created through Redshift to determine if they kept their types through the ETL process

 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

The quality checks that will determine whether the pipeline ran successfully includes verifying that the tables located in Redshift are populated **(i.e. the function below check_tables)** and verifying that the count of unique keys are the same compared to the initial dataframe created prior to their final location in **SAS_dimensions_cleaned**.

In [56]:
# Perform quality checks here
import configparser
import psycopg2
from sql_queries import table_loads

def check_tables(cur,conn):
    #Checks the number of rows per table
    for query in table_loads:
        cur.execute(query)
    
    #retrieves the next row of a query result set
        output = cur.fetchone()
    
    #Print the output one line at a time
        for i in output:
            print(' ',i)
            

def main():
    config = configparser.ConfigParser()
    config.read('dl.cfg')
    
    conn = psycopg2.connect("host={} dbname={} user={} password ={} port ={}".format(*config['CLUSTER'].values()))
    cur = conn.cursor()
    
    #runs the function check_tables
    check_tables(cur,conn)
    
    conn.close()

    
if __name__ == "__main__":
    main()

  2875
  1058572
  55075
  8235082
  289
  660
  4
  55
  3


In [57]:
redshift_host = "redshift-cluster-1.cudjvg0hvfoh.us-west-2.redshift.amazonaws.com"
redshift_user = "awsuser"
redshift_password = "xxxxx"
port = 5439
dbname = "dev"

In [58]:
#Import create_engine & text to verify the results from the table insertions
from sqlalchemy import create_engine
from sqlalchemy import text
engine_string = "postgresql+psycopg2://%s:%s@%s:%d/%s" \
% (redshift_user, redshift_password, redshift_host, port, dbname)
engine = create_engine(engine_string)

In [69]:
#check the primary keys for the immigration table
sql_immigration = """ SELECT COUNT(DISTINCT cicid) as distinct_cicid from i94_immigration_data"""
check1 = pd.read_sql_query(text(sql_immigration),engine)

In [78]:
unique_cicid = check1['distinct_cicid'][0]

In [79]:
unique_cicid_original = immigration_df['cicid'].nunique()

In [80]:
#Function to compare the data quality of the original csv vs table created through Redshift
def check_unique_counts(table,csv):
    if table == csv:
        print('Primary key values match; This dataset is valid')
    else:
        print('Primary key values do not match; Check the dataset again')

In [81]:
#place the unique counts of the cicid in the check_unique_counts function
check_unique_counts(unique_cicid,unique_cicid_original)

Primary key values match; This dataset is valid


In [82]:
#check primary keys for the airport table
sql_airport_data = """ SELECT COUNT(DISTINCT ident) as distinct_ident from airport_data"""
ident_unique = pd.read_sql_query(text(sql_airport_data),engine)

In [83]:
unique_ident = ident_unique['distinct_ident'][0]

In [84]:
unique_ident_original = airport_codes['ident'].nunique()

In [85]:
#place the unique counts of the ident in the check_unique_counts function
check_unique_counts(unique_ident,unique_ident_original)

Primary key values match; This dataset is valid


In [87]:
sql_i94_cntyl = """ SELECT COUNT(distinct country) as distinct_country from i94_cntyl_data"""
sql_i94_cntyl_result1 = pd.read_sql_query(text(sql_i94_cntyl),engine)

In [88]:
sql_i94_cntyl_result1['distinct_country'][0]

287

In [89]:
#check primary keys for the country table
sql_i94_cntyl = """ SELECT COUNT(distinct code) as country_code_distinct from i94_cntyl_data """
sql_i94_cntyl_result2 = pd.read_sql_query(text(sql_i94_cntyl),engine)

In [90]:
unique_country_code = sql_i94_cntyl_result2['country_code_distinct'][0]

In [93]:
unique_country_code_original = i94cntyl_df['code'].nunique()

In [94]:
#place the unique counts of the country codes in the check_unique_counts function
check_unique_counts(unique_country_code,unique_country_code_original)

Primary key values match; This dataset is valid


In [96]:
#check primary keys for the port table
sql_i94_port = """ SELECT COUNT(distinct code) as port_code_distinct from i94_port_data"""
sql_i94_port_result = pd.read_sql_query(text(sql_i94_port),engine)

In [98]:
unique_port_code = sql_i94_port_result['port_code_distinct'][0]

In [99]:
unique_port_code_original = i94port_df['code'].nunique()

In [100]:
#place the unique counts of the country codes in the check_unique_counts function
check_unique_counts(unique_port_code,unique_port_code_original)

Primary key values match; This dataset is valid


In [101]:
#Check the number of codes match up with the i94_mode_df
sql_i94_mode = """ SELECT COUNT(DISTINCT code) as mode_code_distinct from i94_mode_data"""
sql_i94_mode_result = pd.read_sql_query(text(sql_i94_mode),engine)

In [102]:
unique_mode_code = sql_i94_mode_result['mode_code_distinct'][0]

In [103]:
unique_mode_code_original = i94mode_df['code'].nunique()

In [104]:
#place the unique counts of the mode codes in the check_unique_counts function
check_unique_counts(unique_mode_code,unique_mode_code_original)

Primary key values match; This dataset is valid


In [70]:
#Check the number of codes match up with the addr_df
sql_i94_addr = """ SELECT COUNT(DISTINCT code) from i94_addr_data"""
pd.read_sql_query(text(sql_i94_addr),engine)

,count
0,55


In [71]:
i94addr_df['code'].nunique()

55

In [72]:
sql_i94_visa = """ SELECT COUNT(DISTINCT code) from i94_visa_data"""
pd.read_sql_query(text(sql_i94_visa),engine)

,count
0,3


In [73]:
i94_visa_df['value'].nunique()

3

In [75]:
sql_immigration = """ SELECT COUNT(DISTINCT count) as distinct_cicid from i94_immigration_data"""
pd.read_sql_query(text(sql_immigration),engine)

,distinct_cicid
0,1


In [81]:
if airport_codes['ident'].nunique() == len(airport_codes):
    print('city codes are unique')
else:
    print("city codes aren't unique")

city codes are unique


In [84]:
if immigration_df['cicid'].nunique() == immigration_df.cicid.count():
    print('cicids are unique')
else:
    print('cicids arent unique')

cicids are unique


In [87]:
#Check that types match up to original dataframe
check_types_1 = """SELECT
    column_name,
    data_type
FROM
    information_schema.columns
WHERE
    table_name = 'airport_data';"""
pd.read_sql_query(text(check_types_1),engine)

,column_name,data_type
0,local_code,character varying
1,iata_code,character varying
2,gps_code,character varying
3,municipality,character varying
4,iso_region,character varying
5,iso_country,character varying
6,continent,character varying
7,elevation_ft,character varying
8,name,character varying
9,type,character varying


In [88]:
airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55075 entries, 0 to 55074
Data columns (total 13 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       55075 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       55075 non-null object
local_code      28686 non-null object
latitude        55075 non-null float64
longitude       55075 non-null float64
dtypes: float64(3), object(10)
memory usage: 5.9+ MB


In [89]:
#Check that types match up to original dataframe
check_types_2 = """SELECT
    column_name,
    data_type
FROM
    information_schema.columns
WHERE
    table_name = 'us_demographic_data';"""
pd.read_sql_query(text(check_types_2),engine)

,column_name,data_type
0,race,character varying
1,state_code,character varying
2,state,character varying
3,city,character varying
4,avg_household_size,double precision
5,foreign_born,double precision
6,num_of_vets,double precision
7,female_population,double precision
8,male_population,double precision
9,median_age,double precision


In [91]:
us_demographics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2875 entries, 0 to 2890
Data columns (total 12 columns):
City                      2875 non-null object
State                     2875 non-null object
Median Age                2875 non-null float64
Male Population           2875 non-null float64
Female Population         2875 non-null float64
Total Population          2875 non-null int64
Number of Veterans        2875 non-null float64
Foreign-born              2875 non-null float64
Average Household Size    2875 non-null float64
State Code                2875 non-null object
Race                      2875 non-null object
Count                     2875 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 292.0+ KB


**Verify that the ETL has processed the result in the final data model**

In [66]:
#Verify that the ETL processes allow for a combination of tables for analysis
# Part 1
sql_immigration_final = """ SELECT DISTINCT cicid,
                            i94yr,
                            i94mon,
                            i94cit,
                            i94res,
                            i94port,
                            arrdate,
                            i94mode,
                            i94addr,
                            depdate,
                            i94bir,
                            i94visa,
                            count,
                            visapost,
                            biryear,
                            gender,
                            airline,
                            visatype,
                            b.port,
                            c.elevation_ft
                            FROM i94_immigration_data as A
                            LEFT JOIN i94_port_data as B
                            ON A.i94port = B.code
                            LEFT JOIN airport_data C
                            ON A.i94port = C.iata_code
                            WHERE A.i94port = 'ATL' 
                            LIMIT 10"""
pd.read_sql_query(text(sql_immigration_final),engine)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,visapost,biryear,gender,airline,visatype,port,elevation_ft
0,5985165.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,GU,20554.0,6.0,2.0,1.0,SEO,2010.0,M,KE,GMT,"ATLANTA, GA",1026.0
1,5957854.0,2016.0,4.0,254.0,276.0,ATL,20545.0,1.0,GU,20548.0,38.0,2.0,1.0,SEO,1978.0,M,TW,GMT,"ATLANTA, GA",1026.0
2,5900018.0,2016.0,4.0,746.0,158.0,ATL,20574.0,1.0,PA,20590.0,42.0,1.0,1.0,MOS,1974.0,M,AF,B1,"ATLANTA, GA",1026.0
3,5900016.0,2016.0,4.0,746.0,158.0,ATL,20574.0,1.0,GA,20590.0,48.0,1.0,1.0,MOS,1968.0,M,AF,B1,"ATLANTA, GA",1026.0
4,5900014.0,2016.0,4.0,746.0,158.0,ATL,20574.0,1.0,GA,20588.0,49.0,2.0,1.0,MOS,1967.0,F,DL,B2,"ATLANTA, GA",1026.0
5,5899570.0,2016.0,4.0,745.0,745.0,ATL,20574.0,1.0,GA,20603.0,57.0,2.0,1.0,BLG,1959.0,F,AF,B2,"ATLANTA, GA",1026.0
6,5897921.0,2016.0,4.0,696.0,696.0,ATL,20574.0,1.0,GA,20589.0,60.0,2.0,1.0,CRS,1956.0,M,DL,B2,"ATLANTA, GA",1026.0
7,5896273.0,2016.0,4.0,694.0,694.0,ATL,20574.0,1.0,HI,20582.0,48.0,2.0,1.0,LMA,1968.0,M,DL,B2,"ATLANTA, GA",1026.0
8,5896271.0,2016.0,4.0,694.0,694.0,ATL,20574.0,1.0,HI,20580.0,45.0,2.0,1.0,LMA,1971.0,F,DL,B2,"ATLANTA, GA",1026.0
9,5896268.0,2016.0,4.0,694.0,694.0,ATL,20574.0,1.0,GA,20587.0,63.0,2.0,1.0,LMA,1953.0,F,DL,B2,"ATLANTA, GA",1026.0


In [73]:
#Verify that the ETL processes allow for a combination of tables for analysis
# Part 2
sql_immigration_final_2 = """ SELECT DISTINCT cicid,
                            i94yr,
                            i94mon,
                            i94cit,
                            i94res,
                            i94port,
                            arrdate,
                            i94mode,
                            i94addr,
                            depdate,
                            B.type,
                            B.municipality,
                            C.addr as State_Territory
                            FROM i94_immigration_data as A
                            LEFT JOIN airport_data B
                            ON A.i94port = B.iata_code
                            LEFT JOIN i94_addr_data C
                            ON A.i94addr = c.code
                            WHERE c.code = 'TX' and A.i94port = 'DAL' 
                            LIMIT 10"""
pd.read_sql_query(text(sql_immigration_final_2),engine)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,type,municipality,state_territory
0,5900056.0,2016.0,4.0,746.0,158.0,DAL,20574.0,1.0,TX,20588.0,large_airport,Dallas,TEXAS
1,5897934.0,2016.0,4.0,696.0,696.0,DAL,20574.0,1.0,TX,20581.0,large_airport,Dallas,TEXAS
2,5896319.0,2016.0,4.0,694.0,694.0,DAL,20574.0,1.0,TX,20579.0,large_airport,Dallas,TEXAS
3,5896071.0,2016.0,4.0,694.0,582.0,DAL,20574.0,1.0,TX,20603.0,large_airport,Dallas,TEXAS
4,5895362.0,2016.0,4.0,692.0,692.0,DAL,20574.0,1.0,TX,20580.0,large_airport,Dallas,TEXAS
5,5894550.0,2016.0,4.0,691.0,692.0,DAL,20574.0,1.0,TX,20580.0,large_airport,Dallas,TEXAS
6,5892635.0,2016.0,4.0,691.0,691.0,DAL,20574.0,1.0,TX,20587.0,large_airport,Dallas,TEXAS
7,5892633.0,2016.0,4.0,691.0,691.0,DAL,20574.0,1.0,TX,20581.0,large_airport,Dallas,TEXAS
8,5892629.0,2016.0,4.0,691.0,691.0,DAL,20574.0,1.0,TX,20581.0,large_airport,Dallas,TEXAS
9,5892627.0,2016.0,4.0,691.0,691.0,DAL,20574.0,1.0,TX,20580.0,large_airport,Dallas,TEXAS


#### 4.3 Data dictionary 


**Dimension table : global_temp_data**
- dt: Date that the average temperature was recorded | VARCHAR
- AverageTemperature: Average temperature of the day | FLOAT
- AverageTemperatureUncertainty: Average uncertainty of temperature | FLOAT
- City: City name of the record | VARCHAR
- Country: Country name of the record | VARCHAR
- Longitude: Latitude of the city | FLOAT
- Latitude: Longitude of the city | FLOAT

**Dimension table : us_demographic_data**
- city: City name used to represent the corresponding demographic information | VARCHAR
- state: State name of the city | VARCHAR
- median_age: Median age of the city | FLOAT
- male_population: Total count of male population of the city | FLOAT
- female_population: Total count of the female population of the city | FLOAT
- total_population: Total population of the city | INTEGER
- foreign_born: Total foreign born population of the city | FLOAT
- avg_household_size: Average household size of the city | FLOAT
- state_code: State code of the city | VARCHAR | Foreign key to code from i94_addr_data
- race: Category of individual in the record | VARCHAR
- count: Record count of the population by race | INTEGER

**Dimension table : airport_data**
- ident: Identify the corresponding airport code used in the immigration data | VARCHAR PRIMARY KEY
- type: Type of airport | VARCHAR
- name: Name of the airport | VARCHAR
- elevation_ft: Elevation of the airport | VARCHAR
- continent: Continent of the airport | VARCHAR
- iso_country: ISO Country code of the airport | VARCHAR
- iso_region: ISO Region code of the airport | VARCHAR
- municipality: Municipality of the airport | VARCHAR
- gps_code: GPS Code of the airport | VARCHAR
- iata_code: IATA Code used to determine the name of the airport | VARCHAR
- local_code: Local code of the airport | VARCHAR
- latitude: Latitude of the airport | FLOAT
- longitude: Longitude of the airport | FLOAT


**Dimension table : i94_addr_data**
- code: Code that corresponds to the state or territory code | VARCHAR PRIMARY KEY
- addr: Name of the state of territory code | VARCHAR

**Dimension table : i94_mode_data**
- code: Code that corresponds to the mode of transportation | VARCHAR PRIMARY KEY
- mode: Name of the mode of transportation | VARCHAR

**Dimension table : i94_port_data**
- code:  Code that maps the city/country/location | VARCHAR PRIMARY KEY
- port: Name of the port that maps city/country/location | VARCHAR

**Dimension table : i94_cntyl_data**
- code: Maps the country code back to the i94_immigration_data table | VARCHAR PRIMARY KEY
- country: Name of the country code | VARCHAR

**Dimension table : i94_visa_data**
- code: Primary key that is used to determine type of visa code | VARCHAR PRIMARY KEY
- type: Type of visa | VARCHAR

**Fact table : i94_immigration_data**
- cicid: ID | VARCHAR
- i94yr: year | VARCHAR
- i94mon: Numeric month | VARCHAR
- i94cit: Cities code | VARCHAR
- i94res: Countries code | VARCHAR | Foreign key to code from i94_cntyl_data
- i94port: Ports code | VARCHAR | Foreign key to code from i94_port_data
- arrdate: Arrival date | VARCHAR
- i94mode: Mode of transportation code | VARCHAR | Foreign key to code from i94_mode_data
- i94addr: State or territory code | VARCHAR | Foreign key to code from i94_addr_data
- depdate: Departure date | VARCHAR
- i94bir: Age | VARCHAR
- i94visa: Visa count | VARCHAR | Foreign key to code from i94_visa_data
- count: Record count | VARCHAR
- visapost : Visa post | VARCHAR
- biryear : Birth year | VARCHAR
- gender : Gender | VARCHAR
- airline : Airline type | VARCHAR
- visatype : Visa type | VARCHAR


#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.

My choice of tools being heavily focused on utilizing the pandas library in Python helped make data exploration and transformation possible in my project.

The decision I made was to do as much of the heavy lifting in the beginning through cleaning and transforming the dataframes, allowing a smooth transition to create the tables and load the data from S3 to Redshift.

Spark was utilized in the background processes such as reading the SAS data in the initial exploration.

* Propose how often the data should be updated and why.

The data would be best updated on a monthly basis to ensure it aligns with the fact table **i94_immigration_data**. This would alleviate the workload required to moving the data into S3 on a daily or weekly basis.

* Write a description of how you would approach the problem differently under the following scenarios:

 * The data was increased by 100x.
 
    - The scenario would require a change of tools, with the option of utilizing Spark or an Amazon EMR instance for parallel processing capability. This would help speed up ETL processes by distributing them among different nodes and simplify the process to allow for scaling up or down depending on the size of the data.
 
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
    - The scenario would be best fit for Airflow, as it would allow for automatic scheduling and monitoring
 
 * The database needed to be accessed by 100+ people.
    - I believe Redshift would be the appropriate tool in this scenario as it allows easy access to query directly with a useful interface and the least amount of configuration.